In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math

In [ ]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [ ]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,1,0,0,0)].copy()

In [ ]:
def changeTimeframe(pricedata, timeframe_mins):
    newpricedata = pd.DataFrame()
    
    index = 0
    
    while(index < len(pricedata)):
        newcandle = pricedata.iloc[index].copy()
        starttime = newcandle.DateTime
        
        while(True):
            index = index + 1
            if index == len(pricedata):
                break
            
            candle = pricedata.iloc[index].copy()
            
            if candle.DateTime - starttime >= datetime.timedelta(minutes=timeframe_mins):
                break
                
            newcandle.AskClose = candle.AskClose
            newcandle.AskHigh = max(newcandle.AskHigh, candle.AskHigh)
            newcandle.AskLow = min(newcandle.AskLow, candle.AskLow)
            newcandle.BidClose = candle.BidClose
            newcandle.BidHigh = max(newcandle.BidHigh, candle.BidHigh)
            newcandle.BidLow = min(newcandle.BidLow, candle.BidLow)
        
        newpricedata = newpricedata.append(newcandle)
        
    return newpricedata

In [ ]:
class CandlestickChart:
    
    def updateChart(self, placeholder):
        starttime = datetime.datetime.combine(self.startdatewidget.value, datetime.time())
        endtime = datetime.datetime.combine(self.enddatewidget.value, datetime.time()) + datetime.timedelta(days=1)
        timeframe = self.timeframewidget.value
        
        fullpricedata = self.fullpricedata
        chartdata = fullpricedata[fullpricedata['DateTime'] >= starttime]
        chartdata = fullpricedata[fullpricedata['DateTime'] < endtime]
        chartdata = changeTimeframe(chartdata, timeframe).copy()
        
        data = self.chart.data[0]
        data.x = [ii for ii in range(len(chartdata['BidLow']))]
        data.low = chartdata['BidLow']
        data.open = chartdata['BidOpen']
        data.close = chartdata['BidClose']
        data.high = chartdata['BidHigh']
        
        layout = self.chart.layout
        layout.xaxis.tickvals = [ii for ii in data.x if 
                                 chartdata.iloc[ii]['DateTime'].time() == datetime.time(9, 30)]
        xaxisdatetimes = [chartdata.iloc[ii]['DateTime'] for ii in layout.xaxis.tickvals]
        layout.xaxis.ticktext = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]
        
        layout.xaxis.hoverformat = ''
    
    def __init__(self, fullpricedata):
        self.fullpricedata = fullpricedata
        
        data = [ dict(
                type = 'candlestick')
               ]
        layout = dict(
                title = 'Candlestick Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        self.chart = go.FigureWidget( data=data, layout=layout)
        
        # DEFINE WIDGETS AND CALLBACK FUNCTIONS
        self.startdatewidget = ipywidgets.DatePicker(
            description='Start Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.enddatewidget = ipywidgets.DatePicker(
            description='End Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.timeframewidget = ipywidgets.IntText(
            description='Timeframe (minutes)',
            value=1,
            disabled=False
        )
        self.widgets = [self.startdatewidget, self.enddatewidget, self.timeframewidget]
        for widget in self.widgets:
            widget.observe(self.updateChart, names = 'value')
        
        self.widgetsdisplay = ipywidgets.VBox([ipywidgets.HBox([self.startdatewidget, self.enddatewidget]),
                                        self.timeframewidget])
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart, self.widgetsdisplay])
        
        self.updateChart(0)

In [ ]:
candle = CandlestickChart(pricedata)
candle.display